In [1]:
import MySQLdb as db
import pandas as pd
from pandas import DataFrame
# file imports
import credentials

In [2]:
# connects script to customer aws-hosted mysql db
database = db.connect(credentials.host, credentials.user, credentials.password, credentials.db)

# stores the location sample results are stored - update as required.
csvfile = 'C:/Users/bjtur/Documents/Capstone/libras_sample.csv'

# stores a common select statement used to extract and append table headers, then the shipment records themselves
select_columns = ("SELECT id, business_sid, industry, sub_industry, shipper, service_type_description, "
                  + "package_count, weight, year_week, shipment_date, delivery_date, "
                  + "delivery_time, freight_charges, freight_discount_amount, misc_charges, "
                  + "misc_discount_amount, net_charge_amount, zone, sender_city, sender_state, sender_zip, "
                  + "sender_country, recipient_city, recipient_state, recipient_zip, recipient_country")

In [5]:
def db_batch (lower, higher):
    """
    Appends a sample of a batch of records to csvfile
    """
    data = pd.read_sql_query(select_columns + " FROM shipment_details limit {}, {};".format(lower, higher), database)
    data_sample = data.sample(frac=0.1, replace=False)
    with open(csvfile, 'a', newline='\n') as f:
        data_sample.to_csv(f, header=False, index=False, sep = ';')

In [6]:
headers = pd.read_sql_query(select_columns + " FROM shipment_details limit 0;", database)
with open(csvfile, 'a', newline='\n') as f:
    headers.to_csv(f, index=False, sep = ';')

In [23]:
import time
start = time.time()
for index in range(0, 30000000, 1000000): # range intentionally extends beyond the known number of records
    batch_start = time.time()
    try:
        db_batch(index, 100000)
        print("Index {} | Total time: {} min | Batch time: {} sec".format(index,
                                                                          int(round((time.time() - start)/60, 2)),
                                                                          int(round(time.time() - batch_start))))
    except:
        print("============================================")
        print("Total time: {} min".format(int(round((time.time() - start)/60, 2))))
        break

Index 0 | Total time: 0 min | Batch time: 8 sec
Index 1000000 | Total time: 0 min | Batch time: 10 sec
Index 2000000 | Total time: 0 min | Batch time: 17 sec
Index 3000000 | Total time: 0 min | Batch time: 15 sec
Index 4000000 | Total time: 1 min | Batch time: 16 sec
Index 5000000 | Total time: 1 min | Batch time: 19 sec
Index 6000000 | Total time: 1 min | Batch time: 20 sec
Index 7000000 | Total time: 2 min | Batch time: 23 sec
Index 8000000 | Total time: 2 min | Batch time: 25 sec
Index 9000000 | Total time: 2 min | Batch time: 26 sec
Index 10000000 | Total time: 3 min | Batch time: 33 sec
Index 11000000 | Total time: 4 min | Batch time: 29 sec
Index 12000000 | Total time: 4 min | Batch time: 33 sec
Index 13000000 | Total time: 5 min | Batch time: 33 sec
Index 14000000 | Total time: 5 min | Batch time: 34 sec
Index 15000000 | Total time: 6 min | Batch time: 37 sec
Index 16000000 | Total time: 6 min | Batch time: 38 sec
Index 17000000 | Total time: 7 min | Batch time: 43 sec
Index 180

In [3]:
result = pd.read_sql_query("SELECT recipient_state, COUNT(*) FROM shipment_details GROUP BY recipient_state ORDER BY recipient_state;", database)

In [7]:
result.to_csv("recipient_states.csv")